In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

#import lightgbm as lgb
#import xgboost as xgb
import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

# Preprocessing

In [5]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [6]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [7]:
pseudo = pd.read_csv("../input/in-python-tabular-denoising-residual-network/submission_v2.csv")[target]
test_pseudo = pd.concat([test, pseudo], axis=1)
all_pseudo = pd.concat([train, test_pseudo]).reset_index(drop=True)

# Optuna

In [8]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [9]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo[scale_features], y=all_pseudo[target]):
  """
  """
  param_space = {
                  'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'MVS']),
            'loss_function': 'RMSE',
            'eval_metric': 'RMSE',
          #'task_type': 'GPU',
          'max_depth':trial.suggest_int('max_depth', 4, 6),
           'learning_rate':trial.suggest_uniform('learning_rate', 3.18e-2, 4.38e-2),
          'n_estimators':N_ESTIMATORS,
        'max_bin':trial.suggest_int('max_bin', 136, 186),
             'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 17, 29),
         'reg_lambda':trial.suggest_uniform('reg_lambda', 22.9, 28.9),
            'subsample': trial.suggest_uniform('subsample', 0.156, 0.256)
                 }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  ctb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo[scale_features].iloc[trn_idx], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_rn(X_train)

  X_valid, y_valid = all_pseudo[scale_features].iloc[oof_idx], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]

  #start = time.time()
  for inseed in seed_list:
    param_space['random_seed'] = inseed

    model =  ctb.CatBoostRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        use_best_model=True,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )


    ctb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, ctb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [10]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-27 04:11:52,240] A new study created in memory with name: no-name-dd35a75d-2127-46a7-ab81-b5208a2aff76


0:	learn: 6.3298636	test: 7.8580777	best: 7.8580777 (0)	total: 99.1ms	remaining: 16m 31s
1000:	learn: 6.1921795	test: 7.7803414	best: 7.7803414 (1000)	total: 38s	remaining: 5m 41s
2000:	learn: 6.1347310	test: 7.7716960	best: 7.7716812 (1999)	total: 1m 16s	remaining: 5m 4s
3000:	learn: 6.0833382	test: 7.7676065	best: 7.7676065 (3000)	total: 1m 54s	remaining: 4m 26s
4000:	learn: 6.0353850	test: 7.7636140	best: 7.7635490 (3987)	total: 2m 32s	remaining: 3m 48s
5000:	learn: 5.9887187	test: 7.7611894	best: 7.7611894 (5000)	total: 3m 10s	remaining: 3m 10s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 7.761032468
bestIteration = 5099

Shrink model to first 5100 iterations.
0:	learn: 6.3297614	test: 7.8579905	best: 7.8579905 (0)	total: 38.8ms	remaining: 6m 28s
1000:	learn: 6.1912456	test: 7.7799068	best: 7.7799068 (1000)	total: 38s	remaining: 5m 41s
2000:	learn: 6.1335463	test: 7.7712244	best: 7.7712244 (2000)	total: 1m 15s	remaining: 5m 3s
3000:	learn: 6.0825520	test: 7.76

[I 2021-08-27 04:18:35,643] Trial 0 finished with value: 7.759287875651061 and parameters: {'bootstrap_type': 'Bernoulli', 'max_depth': 6, 'learning_rate': 0.035419850921570656, 'max_bin': 179, 'min_data_in_leaf': 29, 'reg_lambda': 27.64976072571717, 'subsample': 0.22690964266733676}. Best is trial 0 with value: 7.759287875651061.


0:	learn: 6.3299457	test: 7.8581068	best: 7.8581068 (0)	total: 34.6ms	remaining: 5m 46s
1000:	learn: 6.1971826	test: 7.7824785	best: 7.7824785 (1000)	total: 33.4s	remaining: 5m
2000:	learn: 6.1424355	test: 7.7726008	best: 7.7725620 (1980)	total: 1m 6s	remaining: 4m 26s
3000:	learn: 6.0943620	test: 7.7680684	best: 7.7680270 (2996)	total: 1m 40s	remaining: 3m 53s
4000:	learn: 6.0486227	test: 7.7652846	best: 7.7652643 (3996)	total: 2m 14s	remaining: 3m 21s
5000:	learn: 6.0046966	test: 7.7627231	best: 7.7627167 (4999)	total: 2m 47s	remaining: 2m 47s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 7.761612142
bestIteration = 5397

Shrink model to first 5398 iterations.
0:	learn: 6.3297254	test: 7.8578862	best: 7.8578862 (0)	total: 33.9ms	remaining: 5m 38s
1000:	learn: 6.1975937	test: 7.7821999	best: 7.7821999 (1000)	total: 34.3s	remaining: 5m 8s
2000:	learn: 6.1422683	test: 7.7732213	best: 7.7732148 (1981)	total: 1m 8s	remaining: 4m 33s
3000:	learn: 6.0944216	test: 7.7688

[I 2021-08-27 04:26:48,848] Trial 1 finished with value: 7.758044449058102 and parameters: {'bootstrap_type': 'Bernoulli', 'max_depth': 6, 'learning_rate': 0.033977878786145985, 'max_bin': 136, 'min_data_in_leaf': 28, 'reg_lambda': 23.193878789603225, 'subsample': 0.18022975777354922}. Best is trial 1 with value: 7.758044449058102.


...

0:	learn: 6.3295682	test: 7.8577584	best: 7.8577584 (0)	total: 37.3ms	remaining: 6m 12s
1000:	learn: 6.2022757	test: 7.7792195	best: 7.7792195 (1000)	total: 37s	remaining: 5m 32s
2000:	learn: 6.1557289	test: 7.7700391	best: 7.7700318 (1999)	total: 1m 14s	remaining: 4m 57s
3000:	learn: 6.1159865	test: 7.7657193	best: 7.7656388 (2968)	total: 1m 51s	remaining: 4m 20s
4000:	learn: 6.0783970	test: 7.7630087	best: 7.7629122 (3901)	total: 2m 28s	remaining: 3m 43s
5000:	learn: 6.0425878	test: 7.7604739	best: 7.7604668 (4995)	total: 3m 5s	remaining: 3m 5s
6000:	learn: 6.0079521	test: 7.7588316	best: 7.7588316 (6000)	total: 3m 43s	remaining: 2m 28s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 7.758014121
bestIteration = 6668

Shrink model to first 6669 iterations.
0:	learn: 6.3296250	test: 7.8577623	best: 7.8577623 (0)	total: 36.9ms	remaining: 6m 9s
1000:	learn: 6.2013733	test: 7.7786285	best: 7.7786285 (1000)	total: 37.2s	remaining: 5m 34s
2000:	learn: 6.1545044	test: 7.76

[I 2021-08-27 08:24:10,155] Trial 28 finished with value: 7.754550726264162 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 5, 'learning_rate': 0.03670937882989713, 'max_bin': 175, 'min_data_in_leaf': 24, 'reg_lambda': 27.870313018881056, 'subsample': 0.2092541796179644}. Best is trial 25 with value: 7.751613272080466.


0:	learn: 6.3296799	test: 7.8579683	best: 7.8579683 (0)	total: 46.5ms	remaining: 7m 45s
1000:	learn: 6.1835187	test: 7.7788949	best: 7.7788888 (999)	total: 44s	remaining: 6m 35s
2000:	learn: 6.1213531	test: 7.7719126	best: 7.7719126 (2000)	total: 1m 27s	remaining: 5m 48s
3000:	learn: 6.0642225	test: 7.7682204	best: 7.7682002 (2994)	total: 2m 10s	remaining: 5m 4s
4000:	learn: 6.0095292	test: 7.7652590	best: 7.7652140 (3995)	total: 2m 53s	remaining: 4m 20s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 7.765002845
bestIteration = 4042

Shrink model to first 4043 iterations.
0:	learn: 6.3297039	test: 7.8578826	best: 7.8578826 (0)	total: 44.5ms	remaining: 7m 24s
1000:	learn: 6.1832159	test: 7.7782453	best: 7.7782453 (1000)	total: 43.2s	remaining: 6m 28s
2000:	learn: 6.1207227	test: 7.7691256	best: 7.7691256 (2000)	total: 1m 26s	remaining: 5m 47s
3000:	learn: 6.0638254	test: 7.7648458	best: 7.7648458 (3000)	total: 2m 9s	remaining: 5m 2s
4000:	learn: 6.0101015	test: 7.761

[I 2021-08-27 08:31:41,673] Trial 29 finished with value: 7.759287765374737 and parameters: {'bootstrap_type': 'Bernoulli', 'max_depth': 6, 'learning_rate': 0.04188999106150025, 'max_bin': 178, 'min_data_in_leaf': 21, 'reg_lambda': 28.53218099901339, 'subsample': 0.19371605545799003}. Best is trial 25 with value: 7.751613272080466.


Number of finished trials: 30
Best trial: {'bootstrap_type': 'MVS', 'max_depth': 6, 'learning_rate': 0.03912784985551841, 'max_bin': 182, 'min_data_in_leaf': 24, 'reg_lambda': 27.78810816753238, 'subsample': 0.20777217056723296}


In [11]:
study.best_params

{'bootstrap_type': 'MVS',
 'max_depth': 6,
 'learning_rate': 0.03912784985551841,
 'max_bin': 182,
 'min_data_in_leaf': 24,
 'reg_lambda': 27.78810816753238,
 'subsample': 0.20777217056723296}

# Log

7.768172118850735 row-wise noise ver4

7.776578973194598 random noise ver5

7.757181558352988 no noise ver7

7.756073089003697 no noise ver8 (narrow space)

7.756000258000434 no noise ver9 (narrow space) final

============= No GPU ================

7.753055265479657 no noise ver13

========================================
================ Change pseudo =========

7.754701143620673 no noise ver15

7.753086397239743 no noise ver16 (narrow space)

7.752308355464106 no noise ver17 (narrow space)